# 增量 PCA（Incremental PCA）

## 概述

增量 PCA（Incremental PCA）是一种支持小批量（mini-batch）更新的 PCA 变体，适合处理无法一次性载入内存的大规模数据集。

## 核心特点

- **内存高效**：逐批处理数据，内存占用恒定
- **流式处理**：支持在线学习场景
- **增量更新**：可以使用 `partial_fit` 方法逐批训练

## 适用场景

- 数据集太大无法一次载入内存
- 流式数据处理
- Out-of-core 计算

## 本节内容

1. 增量 PCA 基本使用
2. 批量处理大数据集
3. 使用 NumPy memmap 处理磁盘数据
4. 与标准 PCA 的对比

## 1. 环境准备

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import tempfile
from sklearn.decomposition import IncrementalPCA, PCA
from sklearn.datasets import fetch_openml
from sklearn.metrics import mean_squared_error
import warnings

warnings.filterwarnings('ignore')

# 设置随机种子
np.random.seed(42)

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

## 2. 数据加载

使用 MNIST 数据集进行演示。

In [ ]:
# 加载 MNIST 数据集
print("正在加载 MNIST 数据集...")
mnist = fetch_openml('mnist_784', version=1, as_frame=False, parser='auto')
X = mnist.data.astype(np.float32)
y = mnist.target

# 使用子集进行演示
n_samples = 20000
X = X[:n_samples]
y = y[:n_samples]

# 归一化
X = X / 255.0

print(f"数据形状: {X.shape}")
print(f"数据内存占用: {X.nbytes / 1024 / 1024:.2f} MB")

## 3. 增量 PCA 基本使用

使用 `partial_fit` 方法逐批训练模型。

In [ ]:
# 设置参数
n_components = 154
n_batches = 100
batch_size = n_samples // n_batches

print(f"主成分数: {n_components}")
print(f"批次数: {n_batches}")
print(f"每批样本数: {batch_size}")

In [ ]:
# 创建增量 PCA 模型
inc_pca = IncrementalPCA(n_components=n_components)

# 逐批训练
print("开始增量训练...")
for i, X_batch in enumerate(np.array_split(X, n_batches)):
    inc_pca.partial_fit(X_batch)
    if (i + 1) % 20 == 0:
        print(f"  已处理 {i + 1}/{n_batches} 批次")

print("训练完成！")
print(f"\n保留的方差比: {inc_pca.explained_variance_ratio_.sum():.4f}")

In [ ]:
# 转换数据
X_reduced = inc_pca.transform(X)

print(f"原始数据形状: {X.shape}")
print(f"降维后数据形状: {X_reduced.shape}")
print(f"\n原始数据内存: {X.nbytes / 1024 / 1024:.2f} MB")
print(f"降维后数据内存: {X_reduced.nbytes / 1024 / 1024:.2f} MB")

## 4. 与标准 PCA 对比

比较增量 PCA 和标准 PCA 的结果差异。

In [ ]:
# 标准 PCA（需要将所有数据加载到内存）
std_pca = PCA(n_components=n_components)
X_std_reduced = std_pca.fit_transform(X)

print("解释方差对比:")
print(f"  增量 PCA: {inc_pca.explained_variance_ratio_.sum():.6f}")
print(f"  标准 PCA: {std_pca.explained_variance_ratio_.sum():.6f}")
print(f"  差异: {abs(inc_pca.explained_variance_ratio_.sum() - std_pca.explained_variance_ratio_.sum()):.6f}")

In [ ]:
# 重构误差对比
X_inc_recon = inc_pca.inverse_transform(X_reduced)
X_std_recon = std_pca.inverse_transform(X_std_reduced)

mse_inc = mean_squared_error(X, X_inc_recon)
mse_std = mean_squared_error(X, X_std_recon)

print(f"\n重构 MSE 对比:")
print(f"  增量 PCA: {mse_inc:.6f}")
print(f"  标准 PCA: {mse_std:.6f}")

In [ ]:
# 可视化对比
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 解释方差对比
ax1 = axes[0]
ax1.plot(range(1, n_components + 1), inc_pca.explained_variance_ratio_, 
         'o-', label='Incremental PCA', alpha=0.7, markersize=3)
ax1.plot(range(1, n_components + 1), std_pca.explained_variance_ratio_, 
         's--', label='Standard PCA', alpha=0.7, markersize=3)
ax1.set_xlabel('Principal Component')
ax1.set_ylabel('Explained Variance Ratio')
ax1.set_title('Individual Explained Variance')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 累计方差对比
ax2 = axes[1]
cumsum_inc = np.cumsum(inc_pca.explained_variance_ratio_)
cumsum_std = np.cumsum(std_pca.explained_variance_ratio_)
ax2.plot(range(1, n_components + 1), cumsum_inc, 'o-', 
         label='Incremental PCA', alpha=0.7, markersize=3)
ax2.plot(range(1, n_components + 1), cumsum_std, 's--', 
         label='Standard PCA', alpha=0.7, markersize=3)
ax2.axhline(y=0.95, color='red', linestyle='--', alpha=0.5, label='95% threshold')
ax2.set_xlabel('Number of Components')
ax2.set_ylabel('Cumulative Explained Variance')
ax2.set_title('Cumulative Explained Variance')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.suptitle('Incremental PCA vs Standard PCA', fontsize=14)
plt.tight_layout()
plt.show()

## 5. 重构图像可视化

In [ ]:
# 选择几张图像可视化
n_display = 5
indices = np.random.choice(n_samples, n_display, replace=False)

fig, axes = plt.subplots(3, n_display, figsize=(12, 7))

for i, idx in enumerate(indices):
    # 原始图像
    axes[0, i].imshow(X[idx].reshape(28, 28), cmap='gray')
    axes[0, i].axis('off')
    if i == 0:
        axes[0, i].set_title('Original')
    
    # 增量 PCA 重构
    axes[1, i].imshow(X_inc_recon[idx].reshape(28, 28), cmap='gray')
    axes[1, i].axis('off')
    if i == 0:
        axes[1, i].set_title('Incremental PCA')
    
    # 标准 PCA 重构
    axes[2, i].imshow(X_std_recon[idx].reshape(28, 28), cmap='gray')
    axes[2, i].axis('off')
    if i == 0:
        axes[2, i].set_title('Standard PCA')

plt.suptitle(f'Image Reconstruction (n_components={n_components})', fontsize=14)
plt.tight_layout()
plt.show()

## 6. 使用 NumPy memmap 处理磁盘数据

当数据太大无法载入内存时，可以使用 `numpy.memmap` 将数据存储在磁盘上，按需加载。

In [ ]:
# 创建临时目录
temp_dir = tempfile.mkdtemp()
memmap_file = os.path.join(temp_dir, 'mnist_memmap.dat')

# 将数据写入 memmap 文件
m, n = X.shape
X_mm = np.memmap(memmap_file, dtype='float32', mode='w+', shape=(m, n))
X_mm[:] = X
X_mm.flush()  # 确保数据写入磁盘

print(f"Memmap 文件创建完成: {memmap_file}")
print(f"文件大小: {os.path.getsize(memmap_file) / 1024 / 1024:.2f} MB")

In [ ]:
# 使用 memmap 进行增量 PCA 训练
# 重新打开 memmap 文件（只读模式）
X_mm_read = np.memmap(memmap_file, dtype='float32', mode='r', shape=(m, n))

# 使用 batch_size 参数自动处理批次
batch_size_mm = m // 50
inc_pca_mm = IncrementalPCA(n_components=n_components, batch_size=batch_size_mm)

# 直接在 memmap 上训练（IncrementalPCA 会自动分批处理）
print("在 memmap 数据上训练...")
inc_pca_mm.fit(X_mm_read)
print("训练完成！")

print(f"\n保留的方差比: {inc_pca_mm.explained_variance_ratio_.sum():.4f}")

In [ ]:
# 清理临时文件
del X_mm_read
os.remove(memmap_file)
os.rmdir(temp_dir)
print("临时文件已清理")

## 7. 批次大小的影响

批次大小会影响结果的精度和计算效率。

In [ ]:
# 测试不同的批次大小
batch_sizes = [200, 500, 1000, 2000]
results_bs = []

# 标准 PCA 作为基准（重新获取）
std_variance = std_pca.explained_variance_ratio_.sum()

print("批次大小对结果的影响:")
print("-" * 50)

for bs in batch_sizes:
    inc_pca_test = IncrementalPCA(n_components=n_components)
    
    for X_batch in np.array_split(X, max(1, n_samples // bs)):
        inc_pca_test.partial_fit(X_batch)
    
    variance = inc_pca_test.explained_variance_ratio_.sum()
    diff = abs(variance - std_variance)
    
    results_bs.append({
        'batch_size': bs,
        'variance': variance,
        'diff': diff
    })
    
    print(f"批次大小: {bs:5d} | 方差: {variance:.6f} | 与标准 PCA 差异: {diff:.6f}")

In [ ]:
# 可视化批次大小的影响
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

batch_sizes_plot = [r['batch_size'] for r in results_bs]
variances = [r['variance'] for r in results_bs]
diffs = [r['diff'] for r in results_bs]

# 解释方差
ax1 = axes[0]
ax1.plot(batch_sizes_plot, variances, 'o-', color='steelblue', linewidth=2, markersize=8)
ax1.axhline(y=std_variance, color='red', linestyle='--', label='Standard PCA')
ax1.set_xlabel('Batch Size')
ax1.set_ylabel('Total Explained Variance')
ax1.set_title('Explained Variance vs Batch Size')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 与标准 PCA 的差异
ax2 = axes[1]
ax2.bar(range(len(batch_sizes_plot)), diffs, color='coral', alpha=0.7)
ax2.set_xlabel('Batch Size')
ax2.set_ylabel('Difference from Standard PCA')
ax2.set_title('Accuracy Degradation vs Batch Size')
ax2.set_xticks(range(len(batch_sizes_plot)))
ax2.set_xticklabels(batch_sizes_plot)

plt.tight_layout()
plt.show()

## 8. 主成分可视化

In [ ]:
# 可视化前 20 个主成分
n_show = 20
fig, axes = plt.subplots(4, 5, figsize=(12, 10))

for i, ax in enumerate(axes.flat):
    if i < n_show:
        component = inc_pca.components_[i].reshape(28, 28)
        ax.imshow(component, cmap='RdBu_r')
        ax.set_title(f'PC{i+1}\n({inc_pca.explained_variance_ratio_[i]*100:.1f}%)')
    ax.axis('off')

plt.suptitle('Top 20 Principal Components (Incremental PCA)', fontsize=14)
plt.tight_layout()
plt.show()

## 9. 总结

### 关键要点

1. **使用方法**：
   ```python
   # 方法一：使用 partial_fit 逐批训练
   inc_pca = IncrementalPCA(n_components=k)
   for batch in data_batches:
       inc_pca.partial_fit(batch)
   
   # 方法二：使用 batch_size 参数
   inc_pca = IncrementalPCA(n_components=k, batch_size=bs)
   inc_pca.fit(X)  # 自动分批处理
   ```

2. **适用场景**：
   - 数据无法一次载入内存
   - 流式数据处理
   - 在线学习
   - 需要增量更新模型

3. **批次大小选择**：
   - 较大的批次 → 更接近标准 PCA 结果
   - 较小的批次 → 更省内存，但精度可能降低
   - 建议：批次大小 ≥ 5 × n_components

4. **与 memmap 结合**：
   - 使用 `np.memmap` 存储磁盘数据
   - 实现真正的 out-of-core 计算

### 注意事项

- 批次顺序可能影响结果（数据应尽量随机打乱）
- 首批数据需要足够多的样本来估计初始成分
- 结果是近似的，与标准 PCA 有细微差异